# Mix Federated Learning - Logistic Regression



>The following codes are demos only. It's **NOT for production** due to system security concerns, please **DO NOT** use it directly in production.

This tutorial demonstrates how to do `Logistic Regression` with mix partitioned data.

The following is an example of mix partitioning data.

<img alt="dataframe.png" src="../developer/algorithm/resources/mix_data.png" width="600">

Secretflow supports `Logistic Regression` with mix data. The algorithm combines `H`omomorphic `E`ncryption and secure aggregation for better security, for more details please refer to [Federated Logistic Regression](../developer/algorithm/mix_lr.md)


## Initialization

Init secretflow.

In [1]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob', 'carol', 'dave', 'eric'], address='local', num_cpus=64)

alice, bob, carol, dave, eric = (
    sf.PYU('alice'),
    sf.PYU('bob'),
    sf.PYU('carol'),
    sf.PYU('dave'),
    sf.PYU('eric'),
)

The version of SecretFlow: 1.7.0b0


/home/beng003/anaconda/envs/sf_1.7/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-08-01 15:40:56,342	INFO worker.py:1724 -- Started a local Ray instance.


(raylet) [2024-08-01 15:41:56,382 E 1361065 1361065] (raylet) node_manager.cc:3024: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: b17a725100fa77fdcb0284852df0aed02ea5a7b7220eb47301a9d448, IP: 10.0.0.4) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.0.0.4`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2024-08-01 15:42:56,383 E 1361065 1361065] (raylet) node_manager.cc:3024: 2 Workers (tasks / actors) killed due

## Data Preparation

We use [brease canser](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic)) as our dataset.

Let us build a mix partitioned data with this dataset. The partitions are as follows:

|label|feature1\~feature10|feature11\~feature20|feature21\~feature30|
|---|---|---|--|
|alice_y0|alice_x0|bob_x0|carol_x|
|alice_y1|alice_x1|bob_x1|dave_x|
|alice_y2|alice_x2|bob_x2|eric_x|

Alice holds all label and all 1~10 features, bob holds all 11~20 fetures,
carol/dave/eric hold a part of 21~30 features separately.


In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

features, label = load_breast_cancer(return_X_y=True, as_frame=True)
features.iloc[:, :] = StandardScaler().fit_transform(features)
label = label.to_frame()


feat_list = [
    features.iloc[:, :10],
    features.iloc[:, 10:20],
    features.iloc[:, 20:],
]

alice_y0, alice_y1, alice_y2 = label.iloc[0:1], label.iloc[1:2], label.iloc[2:3]
alice_x0, alice_x1, alice_x2 = (
    feat_list[0].iloc[0:1, :],
    feat_list[0].iloc[1:2, :],
    feat_list[0].iloc[2:3, :],
)
bob_x0, bob_x1, bob_x2 = (
    feat_list[1].iloc[0:1, :],
    feat_list[1].iloc[1:2, :],
    feat_list[1].iloc[2:3, :],
)
carol_x, dave_x, eric_x = (
    feat_list[2].iloc[0:1, :],
    feat_list[2].iloc[1:2, :],
    feat_list[2].iloc[2:3, :],
)

In [3]:
import tempfile

tmp_dir = tempfile.mkdtemp()


def filepath(filename):
    return f'{tmp_dir}/{filename}'


alice_y0_file, alice_y1_file, alice_y2_file = (
    filepath('alice_y0'),
    filepath('alice_y1'),
    filepath('alice_y2'),
)
alice_x0_file, alice_x1_file, alice_x2_file = (
    filepath('alice_x0'),
    filepath('alice_x1'),
    filepath('alice_x2'),
)
bob_x0_file, bob_x1_file, bob_x2_file = (
    filepath('bob_x0'),
    filepath('bob_x1'),
    filepath('bob_x2'),
)
carol_x_file, dave_x_file, eric_x_file = (
    filepath('carol_x'),
    filepath('dave_x'),
    filepath('eric_x'),
)

alice_x0.to_csv(alice_x0_file, index=False)
alice_x1.to_csv(alice_x1_file, index=False)
alice_x2.to_csv(alice_x2_file, index=False)
bob_x0.to_csv(bob_x0_file, index=False)
bob_x1.to_csv(bob_x1_file, index=False)
bob_x2.to_csv(bob_x2_file, index=False)
carol_x.to_csv(carol_x_file, index=False)
dave_x.to_csv(dave_x_file, index=False)
eric_x.to_csv(eric_x_file, index=False)

alice_y0.to_csv(alice_y0_file, index=False)
alice_y1.to_csv(alice_y1_file, index=False)
alice_y2.to_csv(alice_y2_file, index=False)

Now create `MixDataFrame` x and y for further usage.
`MixDataFrame` is a list of `HDataFrame` or `VDataFrame`

> you can read secretflow's [DataFrame](../user_guide/preprocessing/DataFrame.ipynb) for more details.

In [4]:
vdf_x0 = sf.data.vertical.read_csv(
    {alice: alice_x0_file, bob: bob_x0_file, carol: carol_x_file}
)
vdf_x1 = sf.data.vertical.read_csv(
    {alice: alice_x1_file, bob: bob_x1_file, dave: dave_x_file}
)
vdf_x2 = sf.data.vertical.read_csv(
    {alice: alice_x2_file, bob: bob_x2_file, eric: eric_x_file}
)
vdf_y0 = sf.data.vertical.read_csv({alice: alice_y0_file})
vdf_y1 = sf.data.vertical.read_csv({alice: alice_y1_file})
vdf_y2 = sf.data.vertical.read_csv({alice: alice_y2_file})


x = sf.data.mix.MixDataFrame(partitions=[vdf_x0, vdf_x1, vdf_x2])
y = sf.data.mix.MixDataFrame(partitions=[vdf_y0, vdf_y1, vdf_y2])

INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party carol.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party dave.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party eric.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.
INFO:root:Create p

## Training

Construct `HEU` and `SecureAggregator` for further training.

In [5]:
from typing import List

from secretflow.security.aggregation import SecureAggregator
import spu


def heu_config(sk_keeper: str, evaluators: List[str]):
    return {
        'sk_keeper': {'party': sk_keeper},
        'evaluators': [{'party': evaluator} for evaluator in evaluators],
        'mode': 'PHEU',
        'he_parameters': {
            'schema': 'paillier',
            'key_pair': {'generate': {'bit_size': 2048}},
        },
    }


heu0 = sf.HEU(heu_config('alice', ['bob', 'carol']), spu.spu_pb2.FM128)
heu1 = sf.HEU(heu_config('alice', ['bob', 'dave']), spu.spu_pb2.FM128)
heu2 = sf.HEU(heu_config('alice', ['bob', 'eric']), spu.spu_pb2.FM128)
aggregator0 = SecureAggregator(alice, [alice, bob, carol])
aggregator1 = SecureAggregator(alice, [alice, bob, dave])
aggregator2 = SecureAggregator(alice, [alice, bob, eric])

(raylet) WARNING: 32 PYTHON worker processes have been started on node: b17a725100fa77fdcb0284852df0aed02ea5a7b7220eb47301a9d448 with address: 10.0.0.4. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
(raylet) WARNING: 40 PYTHON worker processes have been started on node: b17a725100fa77fdcb0284852df0aed02ea5a7b7220eb47301a9d448 with address: 10.0.0.4. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).


INFO:root:Create proxy actor <class 'secretflow.security.aggregation.secure_aggregator._Masker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.secure_aggregator._Masker'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.secure_aggregator._Masker'> with party carol.


OutOfMemoryError: Task was killed due to the node running low on memory.
Memory on the node (IP: 10.0.0.4, ID: b17a725100fa77fdcb0284852df0aed02ea5a7b7220eb47301a9d448) where the task (task ID: ffffffffffffffff8c0353457ff6e45a59d8ba2e01000000, name=_Masker.__init__, pid=1362246, memory used=0.08GB) was running was 14.38GB / 15.11GB (0.951635), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: b98c7cd8ed949909f445d43393bc3f7ce187c87a6ab72aa2b97a85eb) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 10.0.0.4`. To see the logs of the worker, use `ray logs worker-b98c7cd8ed949909f445d43393bc3f7ce187c87a6ab72aa2b97a85eb*out -ip 10.0.0.4. Top 10 memory users:
PID	MEM(GB)	COMMAND
1321436	0.89	/home/beng003/.vscode-server/cli/servers/Stable-f1e16e1e6214d7c44d078b1f0607b2388f29d729/server/node...
1322060	0.70	/home/beng003/.vscode-server/cli/servers/Stable-f1e16e1e6214d7c44d078b1f06

Fit the  model.

In [6]:
import logging

logging.root.setLevel(level=logging.INFO)

from secretflow.ml.linear import FlLogisticRegressionMix

model = FlLogisticRegressionMix()

model.fit(
    x,
    y,
    batch_size=64,
    epochs=3,
    learning_rate=0.1,
    aggregators=[aggregator0, aggregator1, aggregator2],
    heus=[heu0, heu1, heu2],
)

INFO:root:MixLr epoch 0: loss = 0.22200048124132613
INFO:root:MixLr epoch 1: loss = 0.10997288443236536
INFO:root:MixLr epoch 2: loss = 0.08508413270494121
INFO:root:MixLr epoch 3: loss = 0.07325763613227645


## Prediction

Let us do predictions with the model.

In [7]:
import numpy as np
from sklearn.metrics import roc_auc_score

y_pred = np.concatenate(sf.reveal(model.predict(x)))

auc = roc_auc_score(label.values, y_pred)
acc = np.mean((y_pred > 0.5) == label.values)
print('auc:', auc, ', acc:', acc)

auc: 0.9875535119708261 , acc: 0.9384885764499121


## The end
Clean temp files.

In [8]:
import shutil

shutil.rmtree(tmp_dir, ignore_errors=True)